In [1]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/summary/multi-news-original-20190725T164630Z-001.zip
# !unzip multi-news-original-20190725T164630Z-001.zip

In [16]:
from tqdm import tqdm

files = ['train', 'test', 'val']

results = []

for file in files:
    s = f'multi-news-original/{file}.src'
    t = f'multi-news-original/{file}.tgt'
    
    with open(s) as fopen:
        source = fopen.read().split('\n')
        
    with open(t) as fopen:
        tgt = fopen.read().split('\n')
        
    for i in tqdm(range(len(source))):
        s_i = source[i]
        s_i = s_i.replace('|||||', '')
        t_i = tgt[i]
        s_i = [s.strip() for s in s_i.split('NEWLINE_CHAR') if len(s.strip()) >= 30 \
         and not 'toggle caption' in s.lower()]
        if len(s_i) and len(t_i):
            results.append((s_i, t_i))

100%|██████████| 5623/5623 [00:00<00:00, 8968.58it/s] 


In [17]:
def split(story, highlights, cap = 5000):
    a, s_ = [], ''
    for s in story:
        if len(s_ + ' ' + s) >= 5000:
            a.append(s_)
            s_ = ''
        else:
            s_ = s_ + ' ' + s
    if len(s_):
        a.append(s_)
    a.append(highlights)
    return a

In [19]:
splitted = []
for i in tqdm(range(len(results))):
    splitted.append(split(*results[i]))

100%|██████████| 56216/56216 [00:03<00:00, 17962.43it/s]


In [20]:
import json

batch_size = 5000

for i in range(0, len(splitted), batch_size):
    index = min(i + batch_size, len(splitted))
    x = splitted[i: index]
    
    with open(f'multinews-{i}.json', 'w') as fopen:
        json.dump(x, fopen)